In [ ]:
from bs4 import BeautifulSoup
import requests
import datetime
import time
import pandas as pd
import re
from pprint import pprint
import csv
import pickle
import json
from pprint import pprint

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
headers = ['loan value', 'funded percent', 'country', 'loan term', 'loan reason', 'repayment schedule', 'disbursment date', 
           'exchange loss', 'partner agency', 'field partner', 'interest', 'risk rating']
kiva_info = []

In [ ]:
#applied for access to the developer api, below is my API ID for repeated calls to the API
#app_id = '?ttrumbly.github.com.metis.bootcamp'

In [ ]:
#response = requests.get('http://api.kivaws.org/v1/partners.json?app_id=ttrumbly.github.com.metis.bootcamp')
#response.headers

In [ ]:
def pull_data(url):
    response = requests.get(url, headers={'User-Agent': 'Chrome/64.0'})
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    return soup

In [ ]:
def funded_percent(soup):
    try:
        funded = (soup.find(class_='green-bolded inline')).text.strip()[0]
        if (funded == 'F'):
            return 100
        else:
            return int(funded)
    except: pass

In [ ]:
def create_dict(soup):
    #This calculates the percent funded (0-100)
    funded = funded_percent(soup)
    try:
        #This calculates the total requested value of the loan
        loan = int(soup.find(class_='loan-total').text.strip().split('$')[-1])
        
        #This gets the str for the length of the loan term.
        term = int(soup.find(id='ac-loan-details-body-right').find(class_='green-bolded').text.strip().split(' ')[0])

        #This pulls the country
        locale = (soup.find('div', class_='country-image columns small-2'))
        country = str(locale.find('img')).split()[1].split('"')[1]
        
        #Returns the reason for the loan
        reason = soup.find(class_='typeName').text.strip()
        
        #Gets the loan repayment schedule
        schedule = (soup.find(id='ac-loan-details-body-right').find(id='repayment-schedule').text.strip().split(' ')[-1])

        #Returns the date the loan was disbersed
        disbursed = (soup.find(id='ac-loan-details-body-right').find(id='disbursed-date').text.strip().split(' ')[-3:])
        disbursed = datetime.datetime.strptime(" ".join(disbursed),"%B %d, %Y").date()

        #Returns currency exchange loss possibility
        exchange_loss = (soup.find(id='ac-loan-details-body-right').find(id='currency-exchange-loss').text.strip().split(' ')[-1])

        #Returns name of field partner
        field_partner = (soup.find(id='ac-loan-details-body-right').find(id='field-partner').text.strip().split(' '))
        partner = ' '.join(field_partner[0:4]).split(':')[0]
        field_partner = ' '.join(field_partner[-2:])

        #Returns if loan recipient is paying interest
        interest = (soup.find(id='ac-loan-details-body-right').find(id='borrower-paying-interest').text.strip().split(' ')[-1])

        #Calculates the numeric number of stars representing the risk rating
        risk_stars = 0
        for tag in (soup.find(id='ac-loan-details-body-right').find_all(class_='icon icon-star')):
            risk_stars+=1
        for tag in (soup.find(id='ac-loan-details-body-right').find_all(class_='icon icon-half-star')):
            risk_stars +=.5

        kiva_dict = dict(zip(headers, [loan,
                                       funded,
                                       country,
                                       term,
                                       reason,
                                       schedule,
                                       disbursed,
                                       exchange_loss,
                                       partner,
                                       field_partner,
                                       interest,
                                       risk_stars]))
        return kiva_dict
    except:
        pass

In [ ]:
#locale = (soup.find('div', class_='country-image columns small-2'))
#country = str(locale.find('img')).split()[1].split('"')[1]

In [ ]:
#This loop sequentially goes through a series of kiva sites and calls functions to turn the website into a dictionary entry
number = 1507304
while (number <=1507304):
    baseurl = 'https://www.kiva.org/lend/'
    url = baseurl + str(number)
    print (url)
    number+=1
    soup = pull_data(url)
    kiva_info.append(create_dict(soup))
    time.sleep(1)

In [ ]:
#Create a dataframe from the entries scraped that are not 'None' entries.
df = pd.DataFrame([x for x in kiva_info if x is not None], columns = headers)

In [ ]:
#Save the dataframe to a pickle for future work
df.to_pickle('project2_df.pkl')

In [ ]:
df2= pd.read_pickle('project2_df.pkl')

In [ ]:
from urllib.request import urlopen

In [ ]:
data = json.load(urlopen(json_url))

In [ ]:
df_3 = pd.DataFrame.from_dict(data, orient='index')
#df_3 = pd.read_json(urlopen(json_url), orient='columns', typ='frame')

In [ ]:
def pull_data(url):
    response = requests.get(url, headers={'User-Agent': 'Chrome/65.0'})
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    return soup

In [ ]:
json_url = 'http://api.kivaws.org/v1/loans/1507200.json?ttrumbly.github.com.metis.bootcamp'

In [ ]:
json_file = pull_data(json_url)

In [ ]:
type(json_file)

In [ ]:
data = json.loads(''.join(str(json_file).split('>')[3:-3]))

In [ ]:
number = 1507200

while (number <=1507300):
    url = baseurl + str(number) + app_id
    print (url)
    number+=1
    soup = pull_data(url)
    
    kiva_info.append(create_dict(soup))
    time.sleep(2)
    

In [1]:
#Below uses the data directly from kiva's dataset file. Better seen in Project_2_Kiva_Full_Dataset

In [ ]:
kiva = pd.read_csv('/Users/travis/Downloads/kiva_ds_csv/loans.csv')

In [ ]:
kiva.drop(['ORIGINAL_LANGUAGE', 'DESCRIPTION', 'DESCRIPTION_TRANSLATED', 'IMAGE_ID', 'VIDEO_ID', 'TOWN_NAME', 'TAGS', 'BORROWER_NAMES', 'BORROWER_PICTURED', 'LOAN_USE', 'COUNTRY_CODE'], axis=1, inplace=True)

In [ ]:
#kiva.drop(['LOAN_USE', 'COUNTRY_CODE'], axis=1, inplace= True)
#kiva.drop(['COUNTRY_CODE'], axis=1, inplace=True)

In [ ]:
kiva.describe()

In [ ]:
kiva.info()

In [ ]:
kiva.sample(50)

In [ ]:
kiva[(kiva['LOAN_AMOUNT'] > kiva['FUNDED_AMOUNT']) & (kiva['DISTRIBUTION_MODEL'] == 'direct')]

In [ ]:
kiva['COUNTRY_NAME'].value_counts()

In [ ]:
f, ax = plt.subplots(figsize=(12,12))
sns.countplot(x='COUNTRY_NAME', data=kiva)

In [ ]:
#kiva.to_pickle('kiva.pkl')